In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import numpy as np
from math import sqrt

In [2]:
#target untuk di regresi: RR, menggunakan data tanggal, RH average, SS, dan T average
data = pd.read_csv('data_bmkg_clean.csv',index_col=0)
data

,Tanggal,RH_avg,Tavg,RR,ss
0,01-01-2012,25.1,87.0,39.5,2.9
1,02-01-2012,25.3,85.0,8.5,3.0
2,03-01-2012,26.4,88.0,2.0,3.1
3,04-01-2012,26.7,81.0,2.8,7.8
4,05-01-2012,26.6,85.0,8.8,5.2
...,...,...,...,...,...
3952,26-09-2022,27.4,82.0,9.3,6.0
3953,27-09-2022,27.4,82.0,34.2,3.3
3954,28-09-2022,26.9,81.0,9.9,7.3
3955,29-09-2022,25.8,79.0,2.6,3.4


In [38]:
bulan = 1
day = 0.0
month_as_index = []
RH_avg_monthly = []
Tavg_monthly = []
RR_monthly = []
ss_monthly = []
RH = 0
T = 0
RR = 0
ss = 0 
indeks = 0

def aggr_reset():
        global indeks,ss,RR,T,RH,day
        month_as_index.append(indeks)
        indeks += 1 
        RH_avg_monthly.append(RH/day)
        Tavg_monthly.append(T/day)
        RR_monthly.append(RR/day)
        ss_monthly.append(ss/day)
        day = 0.0
        RH = 0
        T = 0
        RR = 0
        ss = 0 

In [39]:


for index in data.index:
    date = data['Tanggal'][index]
    mo = int(date.split("-")[1])
    if mo == 1 and bulan == 12:
        bulan=1
        aggr_reset()
    if mo == bulan:
        day += 1
        RH = RH+data['RH_avg'][index]
        T = T+data['Tavg'][index]
        RR = RR+data['RR'][index]
        ss = ss+data['ss'][index]
    if mo == bulan+1:
        bulan +=1
        aggr_reset()

In [46]:
monthly_data = {'month_as_index':month_as_index,
                'RH_avg_monthly':RH_avg_monthly,
                'Tavg_monthly':Tavg_monthly,
                'RR_monthly':RR_monthly,
                'ss_monthly':ss_monthly}
monthly_data = pd.DataFrame(monthly_data, index=monthly_data['month_as_index'])
monthly_data

,month_as_index,RH_avg_monthly,Tavg_monthly,RR_monthly,ss_monthly
0,0,26.116129,83.548387,10.987097,3.593548
1,1,26.407143,84.250000,17.757143,4.152168
2,2,26.680000,80.066667,7.370000,4.810000
3,3,26.248276,85.965517,21.558621,4.187192
4,4,26.820000,82.233333,7.636667,5.780000
...,...,...,...,...,...
123,123,6.506897,21.596552,1.806897,2.486207
124,124,4.220000,14.693333,1.313333,6.133333
125,125,4.406897,13.400000,4.217241,6.334483
126,126,27.030000,82.000000,9.953333,4.140000


In [48]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df


monthly_data = swap_columns(monthly_data, 'RR_monthly', 'ss_monthly')

In [49]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [50]:
values = monthly_data[["RH_avg_monthly","Tavg_monthly","RR_monthly","ss_monthly"]].values
# integer encode direction
# encoder = LabelEncoder()
# values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled

array([[0.922026  , 0.9255907 , 0.38769075, 0.49105504],
       [0.9333071 , 0.93402153, 0.6363088 , 0.57236797],
       [0.94388443, 0.8837531 , 0.25485927, 0.6681222 ],
       [0.92714864, 0.9546357 , 0.77591133, 0.577466  ],
       [0.94931155, 0.9097885 , 0.26465213, 0.8093158 ],
       [0.94292194, 0.8721788 , 0.14490482, 0.812227  ],
       [0.9299291 , 0.8364889 , 0.07650674, 0.8607471 ],
       [0.9393619 , 0.80244267, 0.37151676, 0.8811256 ],
       [0.99999994, 0.8145833 , 0.07019204, 0.8965516 ],
       [0.950216  , 0.8869575 , 0.5448504 , 0.73508   ],
       [0.9209998 , 0.9728674 , 0.52543354, 0.46890524],
       [0.93974954, 0.93101746, 0.43945473, 0.5263464 ],
       [0.9161487 , 0.9659036 , 0.6832536 , 0.33798185],
       [0.9305895 , 0.9461492 , 0.40856642, 0.47635987],
       [0.950216  , 0.91259235, 0.5630896 , 0.7045123 ],
       [0.96070033, 0.92148715, 0.40690613, 0.7359333 ],
       [0.946598  , 0.95024353, 0.41472775, 0.5958272 ],
       [0.9431893 , 0.9198297 ,

In [51]:
import pickle 
object = scaler
filehandler = open('scaler_monthly.obj', 'wb') 
pickle.dump(object, filehandler)

In [52]:
window_size = 12
batch_size = 16
shuffle_buffer_size = 1000

In [53]:
train_set = windowed_dataset(scaled,
        window_size=window_size, 
        batch_size=batch_size,
        shuffle_buffer=shuffle_buffer_size)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-11 12:56:07.935650: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-11 12:56:07.937179: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [60]:
train_set

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 4, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 4, 1), dtype=tf.float32, name=None))>

In [64]:
# design network

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(50, input_shape=[None,4]))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mse', optimizer='adam')
# fit network
history = model.fit(train_set,
                    epochs=50, 
                    batch_size=16, 
                    #validation_data=(test_X, test_y), 
                    shuffle=False)
# plot history


Epoch 1/50


2022-12-11 13:05:31.159721: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-11 13:05:31.275618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


InternalError: Graph execution error:

Detected at node 'mean_squared_error/SquaredDifference' defined at (most recent call last):
    File "/Users/iganarendra/miniconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/iganarendra/miniconda3/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/iganarendra/miniconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/iganarendra/miniconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/iganarendra/miniconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/iganarendra/miniconda3/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/Users/